In [ ]:
#pip install python-gnupg
#%pip install azure-storage-file-datalake
#%restart_python
import gnupg
import subprocess
from azure.storage.filedatalake import DataLakeServiceClient
from io import BytesIO
from datetime import datetime

In [ ]:
# Replace with your actual account name and key or other credentials
account_name = "qambusa"
account_key = "F6Cxa90BGhRifHiFcxntdjM0yPm/Wsj3AFZedZJrpMimkHx/27ddUYwAdY5yZGv8XpYhzzjXQm7m+AStN8GtJA=="
file_system_name = "clicmotion" # This is your container name
directory_name = "test/encrypted_keys" # Optional, if writing to a directory
public_key_path = "public_key.asc"
private_key_path = "private_key.asc"

# 1. Authenticate and Create a Service Client
service_client = DataLakeServiceClient(account_url=f"https://{account_name}.dfs.core.windows.net", credential=account_key)

# 2. Get a File System Client
file_system_client = service_client.get_file_system_client(file_system=file_system_name)

# 3. Get a Directory Client (Optional)
directory_client = file_system_client.get_directory_client(directory_name)

# directory_client.create_file(public_key_path)
# directory_client.create_file(private_key_path)

In [ ]:
import subprocess
import os
import tempfile

# Create isolated GPG home
gpg_home = "/tmp/gpg_home"
os.makedirs(gpg_home, exist_ok=True)

env = os.environ.copy()
env["GNUPGHOME"] = gpg_home

# Define key parameters
key_input = """
     %echo Generating GPG key
     Key-Type: RSA
     Key-Length: 4096
     Subkey-Type: RSA
     Subkey-Length: 4096
     Name-Real: Databricks Pipeline
     Name-Email: databricks@example.com
     Expire-Date: 0
     Passphrase: my_secret_passphrase
     %commit
     %echo Done
"""

# Write input to temp file
temp_path = "/tmp/gpg_key_input"
with open(temp_path, "w") as f:
    f.write(key_input)

# Generate key
process = subprocess.run(
    ["gpg", "--batch", "--generate-key", temp_path],
    env=env,
    capture_output=True,
    text=True
)

print(process.stdout, process.stderr)


In [ ]:
# Export public key
pub = subprocess.run(["gpg", "--armor", "--export", "databricks@example.com"], env=env, capture_output=True)
file_client_public = directory_client.get_file_client(public_key_path)
file_client_public.upload_data(pub.stdout, overwrite=True)
print(f"Successfully wrote to file '{public_key_path}' in ADLS Gen2.")

# Export private key
priv = subprocess.run(["gpg",
                       "--pinentry-mode", "loopback",
                       "--passphrase", "my_secret_passphrase",
                       "--armor", 
                       "--export-secret-key", 
                       "databricks@example.com"], env=env,capture_output=True)
file_client_private = directory_client.get_file_client(private_key_path)
file_client_private.upload_data(priv.stdout, overwrite=True) # overwrite=True if you want to replace existing content
print(f"Successfully wrote to file '{private_key_path}' in ADLS Gen2.")